<a href="https://colab.research.google.com/github/Misha-private/Demo-repo/blob/main/DeepNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "How to Build a Deep Neural Network Withour a Framework"

Marco Peixeriro, Medium


### Step 1: Initialize the weights and biases

In [13]:
def initialize_parameters_deep(layer_dims):

  parameters = {}
  L = len(layer_dims) # number of layers in the network

  for l in range(1, L):
    parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
    parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
    assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

  return parameters

### Step 2: Forward propagation model

In [14]:
def linear_forward(A, W, b):

  Z = W.dot(A) + b

  assert(Z.shape == (W.shape[0], A.shape[1]))
  cache = (A, W, b)

  return Z, cache

### Build a function to feed the results to an activation function

(either ReLU for the hidden layers or a sigmoid for the output layer)

In [15]:
def linear_activation_forward(A_prev, W, b, activation):

  if activation == "sigmoid":
    Z, linear_cache = linear_forward(A_prev, W, b)
    A, activation_cache = sigmoid(Z)

  elif activation == "relu":
    Z, linear_cache = linear_forward(A_prev, W, b)
    A, activation_cache = relu(Z)

  assert (A.shape == (W.shape[0], A_prev.shape[1]))
  cache = (linear_cache, activation_cache)

  return A, cache

In [16]:
from os import XATTR_REPLACE
def L_model_forward(X, parameters):

  caches = []
  A = X
  L = len(parameters) // 2 # number of layers in the neural network

  for l in range(1, L):
    A_prev = A
    A, cache = linear_activation_forward(A_prev,
                                         parameters['W' + str(l)],
                                         parameters['b' + str(l)],
                                         activatition ='relu')
    caches.apend(cache)

  AL, cache = linear_activation_forward(A,
                                        parameters['W' + str(L)],
                                        parameters['b' + str(L)],
                                        activation = 'sigmoid')
  caches.append(cache)

  assert(AL.shape == (1, X.shape[1]))

  return AL, caches



### Step 3: Define the cost function

Cross-entropy loss expressed as:

$J = -\frac{1}{m}\sum_{i=1}^m{y_i\log{\hat{y_i}}+(1-y_i)\log{(1-\hat{y_i}})}$


In [17]:
def compute_cost(AL, Y):
  m = Y.shape[1]

  cost = (1./m) * (-np.dot(Y, np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
  cost = np.squeeze(cost)

  assert(cost.shape == ())

  return cost

### Step 4: Backpropagation

In [18]:
def linear_backward(dZ, cache):

  A_prev, W, b = cache
  m = A_prev.shape[1]

  dW = (1./m) * np.dot(dZ, cache[0].T)
  db = (1./m) * np.sum(dZ, axis=1, keepdims=True)
  dA_prev = np.dot(cache[1].T, dZ)

  assert (dA_prev.shape == A_prev.shape)
  assert (dW.shape == W.shape)
  assert (db.shape == b.shape)

  return dA_prev, dW, db



In [19]:
def linear_activation_backward(dA, cache, activation):

  linear_cache, activation_cache = cache

  if activation == "relu":
    dZ = relu_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

  elif activation == "sigmoid":
    dZ = sigmoid_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

  return dA_prev, dW, db

In [20]:
from posixpath import curdir
def L_model_backward(AL, Y, caches):

  grads = {}
  L = len(caches)
  m = AL.shape[1]
  Y = Y.reshape(AL.shape)  # after this line, Y is the same shape as AL

  dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

  current_cache = caches[L-1]
  grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")

  for l in reversed(range(L-1)):
    current_cache = caches[l]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l + 1)] = dW_temp
    grads["db" + str(l + 1)] = db_temp

  return grads

### Step 5:  Update parameters with gradient descent

In [21]:
def update_parameters(parameters, grads, learning_rate):

  L = len(parameters) // 2 # number of layers in the neural network

  for l in range(L):
    parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
    parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]

  return parameters



### Using the neural network

In [22]:
# CONSTANTS
layers_dims = [12288, 20, 7, 5, 1] #  4-layer model

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009

    np.random.seed(1)
    costs = []    # keep track of cost

    parameters = initialize_parameters_deep(layers_dims)

    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)

        # Compute cost
        cost = compute_cost(AL, Y)

        # Backward propagation
        grads = L_model_backward(AL, Y, caches)

        # Update parameters
        parameters = update_parameters(parameters, grads, learning_rate)

        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)

    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    return parameters

# Print training progess and show loss graph
parameters = L_layer_model(train_x, train_y, layers_dims, num_iterations = 2500, print_cost = True)

# Print train accurracy
pred_train = predict(train_x, train_y, parameters)

# Print test accuracy
pred_test = predict(test_x, test_y, parameters)

NameError: name 'train_x' is not defined

#### Use previous model for data and libraries
https://towardsdatascience.com/step-by-step-guide-to-building-your-own-neural-network-from-scratch-df64b1c5ab6e